<a href="https://colab.research.google.com/github/zak-soussi/Football_Match_Prediction/blob/main/xG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
import pandas as pd

# Load the CSV file into a Pandas DataFrame
df = pd.read_csv('shots_bundesliga_14-15.csv')

# View the first few rows of the DataFrame
print("First few rows of the dataset:")
print(df.head())

# View the last few rows
#print("Last few rows of the dataset:")
#print(df.tail())


First few rows of the dataset:
        id  minute       result      X      Y        xG               player  \
0  30358.0     5.0  MissedShots  0.923  0.464  0.141077           Leroy Sané   
1  30359.0     6.0  BlockedShot  0.764  0.427  0.012746     Roman Neustädter   
2  30360.0     8.0         Goal  0.929  0.490  0.530869  Klaas-Jan Huntelaar   
3  30362.0    13.0  MissedShots  0.829  0.555  0.075300          Dennis Aogo   
4  30363.0    14.0    SavedShot  0.889  0.610  0.232375           Leroy Sané   

  h_a  player_id situation  season   shotType  match_id      h_team  \
0   h      337.0  OpenPlay  2014.0   LeftFoot    5159.0  Schalke 04   
1   h      346.0  OpenPlay  2014.0  RightFoot    5159.0  Schalke 04   
2   h      340.0  OpenPlay  2014.0  RightFoot    5159.0  Schalke 04   
3   h      334.0  OpenPlay  2014.0  RightFoot    5159.0  Schalke 04   
4   h      337.0  OpenPlay  2014.0   LeftFoot    5159.0  Schalke 04   

          a_team  h_goals  a_goals                 date  \
0 

In [111]:
#Preprocessing

# Keep only the desired columns
desired_columns = ['minute', 'X', 'Y' , 'h_team', 'a_team', 'situation', 'shotType', 'lastAction' , 'result']
df_subset = df[desired_columns]

# Replace values in the 'result' column
df_subset['result'] = df_subset['result'].replace({'MissedShots': 0, 'BlockedShot': 0, 'SavedShot': 0, 'ShotOnPost': 0, 'OwnGoal': 0, 'Goal': 1})

# Convert the 'result' column to integers
df_subset['result'] = df_subset['result'].astype(int)

# Check for missing values in the entire dataset
missing_values = df_subset.isna().sum()
# Display the number of missing values for each column
print("Missing values per column:")
print(missing_values)


Missing values per column:
minute          0
X               0
Y               0
h_team          0
a_team          0
situation       0
shotType        0
lastAction    900
result          0
dtype: int64


<ipython-input-111-c262eea89c64>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['result'] = df_subset['result'].replace({'MissedShots': 0, 'BlockedShot': 0, 'SavedShot': 0, 'ShotOnPost': 0, 'OwnGoal': 0, 'Goal': 1})
<ipython-input-111-c262eea89c64>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['result'] = df_subset['result'].astype(int)


In [112]:
# Drop rows with NaN values in the 'lastAction' column
df_subset = df_subset.dropna(subset=['lastAction'])


In [113]:
# Check for missing values in the entire dataset
missing_values = df_subset.isna().sum()

# Display the number of missing values for each column
print("Missing values per column:")
print(missing_values)

# Check if there are any missing values in the dataset
if missing_values.sum() == 0:
    print("No missing values found.")
else:
    print("There are missing values in the dataset.")


Missing values per column:
minute        0
X             0
Y             0
h_team        0
a_team        0
situation     0
shotType      0
lastAction    0
result        0
dtype: int64
No missing values found.


In [114]:
#feature engineering

import numpy as np

# Define a function to calculate XDTGi (Distance To Goal)
def calculate_XDTGi(X, Y):
    return np.sqrt((105 - (X * 105))**2 + (32.5 - (Y * 32.5))**2)

# Define a function to calculate XATGi (Angle To Goal)
def calculate_XATGi(X, Y):
    ai = np.arctan(7.32 * (105 - (X * 105)))
    bi = ((105 - (X * 105))**2 + (32.5 - (Y * 68))**2 - (7.32/2)**2)
    return np.abs((ai / bi) * (180 / np.pi))

# Apply the functions to your dataset using .loc
df_subset.loc[:, 'XDTGi'] = calculate_XDTGi(df['X'], df['Y'])
df_subset.loc[:, 'XATGi'] = calculate_XATGi(df['X'], df['Y'])


In [90]:
print(df_subset.head())
print(df_subset.describe())

   minute      X      Y      h_team         a_team situation   shotType  \
0     5.0  0.923  0.464  Schalke 04  VfB Stuttgart  OpenPlay   LeftFoot   
1     6.0  0.764  0.427  Schalke 04  VfB Stuttgart  OpenPlay  RightFoot   
2     8.0  0.929  0.490  Schalke 04  VfB Stuttgart  OpenPlay  RightFoot   
3    13.0  0.829  0.555  Schalke 04  VfB Stuttgart  OpenPlay  RightFoot   
4    14.0  0.889  0.610  Schalke 04  VfB Stuttgart  OpenPlay   LeftFoot   

    lastAction result      XDTGi     XATGi  
0         Pass      0  19.204780  1.683970  
1       Aerial      0  30.997513  0.146387  
2  BlockedPass      1  18.174367  2.075666  
3         Pass      0  23.055279  0.266208  
4       TakeOn      0  17.219019  0.439860  
            minute            X            Y        XDTGi        XATGi
count  7113.000000  7113.000000  7113.000000  7113.000000  7113.000000
mean     48.370730     0.842844     0.503565    24.073515     1.387249
std      26.314495     0.086177     0.138852     7.436806    17.35

In [115]:
from sklearn.model_selection import train_test_split

# Define the features and target variable
X = df_subset.drop('result', axis=1)  # Features
y = df_subset['result']  # Target variable

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the train and test sets
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)


Shape of X_train: (5690, 10)
Shape of X_test: (1423, 10)
Shape of y_train: (5690,)
Shape of y_test: (1423,)


In [116]:
print(y_train.unique())
print(y_train.dtype)

[0 1]
int64


In [83]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 6.7 MB/s eta 0:00:00


In [119]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.preprocessing import LabelEncoder

# Assuming X_train contains categorical variables
label_encoder_X = LabelEncoder()
for column in X_train.columns:
    if X_train[column].dtype == object:
        X_train[column] = label_encoder_X.fit_transform(X_train[column])

# Apply SMOTE to the encoded labels
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train_encoded)
# Apply SMOTE to balance the classes
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Initialize models
rf_model = RandomForestClassifier()
catboost_model = CatBoostClassifier()
xgboost_model = XGBClassifier()
lgbm_model = LGBMClassifier()

# Train models
rf_model.fit(X_train_balanced, y_train_balanced)
catboost_model.fit(X_train_balanced, y_train_balanced)
xgboost_model.fit(X_train_balanced, y_train_balanced)
lgbm_model.fit(X_train_balanced, y_train_balanced)


Learning rate set to 0.027742
0:	learn: 0.6824236	total: 16.3ms	remaining: 16.3s
1:	learn: 0.6717291	total: 27.2ms	remaining: 13.6s
2:	learn: 0.6612824	total: 32.1ms	remaining: 10.7s
3:	learn: 0.6513393	total: 43.5ms	remaining: 10.8s
4:	learn: 0.6417945	total: 50.7ms	remaining: 10.1s
5:	learn: 0.6328595	total: 64.3ms	remaining: 10.7s
6:	learn: 0.6250616	total: 79ms	remaining: 11.2s
7:	learn: 0.6171926	total: 89.4ms	remaining: 11.1s
8:	learn: 0.6058577	total: 101ms	remaining: 11.1s
9:	learn: 0.5982153	total: 113ms	remaining: 11.2s
10:	learn: 0.5916904	total: 125ms	remaining: 11.2s
11:	learn: 0.5858819	total: 129ms	remaining: 10.7s
12:	learn: 0.5797203	total: 136ms	remaining: 10.3s
13:	learn: 0.5743713	total: 141ms	remaining: 9.91s
14:	learn: 0.5688205	total: 146ms	remaining: 9.6s
15:	learn: 0.5632695	total: 152ms	remaining: 9.35s
16:	learn: 0.5584186	total: 158ms	remaining: 9.12s
17:	learn: 0.5526351	total: 164ms	remaining: 8.93s
18:	learn: 0.5486864	total: 169ms	remaining: 8.7s
19:	lea

LGBMClassifier()

In [121]:
# Evaluate models (using validation data or cross-validation)

(X_val, y_val)=(X_test,y_test)
# Assuming X_val contains categorical variables
label_encoder_X = LabelEncoder()
for column in X_val.columns:
    if X_val[column].dtype == object:
        X_val[column] = label_encoder_X.fit_transform(X_val[column])
# Evaluate RandomForestClassifier
rf_accuracy = rf_model.score(X_val, y_val)
# Evaluate CatBoostClassifier
catboost_accuracy = catboost_model.score(X_val, y_val)
# Evaluate XGBClassifier
xgboost_accuracy = xgboost_model.score(X_val, y_val)
# Evaluate LGBMClassifier
lgbm_accuracy = lgbm_model.score(X_val, y_val)

# Print model accuracies
print("RandomForestClassifier Accuracy:", rf_accuracy)
print("CatBoostClassifier Accuracy:", catboost_accuracy)
print("XGBClassifier Accuracy:", xgboost_accuracy)
print("LGBMClassifier Accuracy:", lgbm_accuracy)


RandomForestClassifier Accuracy: 0.8158819395643008
CatBoostClassifier Accuracy: 0.7547434996486296
XGBClassifier Accuracy: 0.7329585382993675
LGBMClassifier Accuracy: 0.7182009838369642
